# Task1_Attention

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_

## Best metric (RMSE)

270.7546443150733



In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.Attention'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='metric',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='metric',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            # ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='metric',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-08-16 12:36:46,203] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-08-16 12:36:48,387	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-16 12:36:49,761	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_01ce1e37,2023-08-16_12-42-16,True,DESKTOP-0P789CI,16,0.4127167741096315,254.02555915207236,0.1043662213762489,3.9002711083112845e+17,0.27587183201561427,172.26.215.93,32121,0.7227230825910833,440.58298521651517,35.543588638305664,2.386256456375122,35.543588638305664,1692157336,0.08427947630654015,0.08974152460848227,14809344365976.217,140493903584120.45,16,01ce1e37,0.13579775851101242,0.14007407350460188
FSR_Trainable_0888e0d9,2023-08-16_12-38-14,,DESKTOP-0P789CI,,,,,,,172.26.215.93,31167,,,,,,1692157094,,,,,,0888e0d9,,
FSR_Trainable_0ac6f2e1,2023-08-16_12-56-25,True,DESKTOP-0P789CI,100,0.4411409142623398,159.4230778694576,0.10160508015392508,1.838216959899216e+17,0.22698495230459997,172.26.215.93,34573,0.7206735267078083,296.2485692107455,236.80179357528687,2.2741830348968506,236.80179357528687,1692158185,0.08855938222245031,0.059136028696270555,13345363572861.066,79275377801636.88,100,0ac6f2e1,0.13351873982628448,0.09346621247831549
FSR_Trainable_12e27424,2023-08-16_12-51-33,True,DESKTOP-0P789CI,2,0.6302932147378268,254.8731181391169,0.1460629189217162,3.364906919060054e+17,0.318580084365253,172.26.215.93,34391,0.9121682530062966,444.4273342451433,4.853215932846069,2.3576855659484863,4.853215932846069,1692157893,0.1248823623639669,0.09628000678387688,16792409442662.883,146270736376022.34,2,12e27424,0.17337046288413022,0.1452096214811228
FSR_Trainable_12e5d895,2023-08-16_12-44-26,True,DESKTOP-0P789CI,16,0.40284502557456764,248.7658639209211,0.10170211317869216,3.6551910702544646e+17,0.27346936312348835,172.26.215.93,32774,0.7148121248727074,437.2415045107349,39.23625373840332,2.4347009658813477,39.23625373840332,1692157466,0.0818557210877971,0.08769497059473758,15214856533088.078,130235611444081.88,16,12e5d895,0.13428690516495684,0.13918245795853154
FSR_Trainable_155143e0,2023-08-16_13-05-13,True,DESKTOP-0P789CI,64,0.4604708923683658,174.88777617851787,0.11143078196960211,2.2628554639125616e+17,0.2387655093240204,172.26.215.93,36075,0.7276543517416573,345.53339542675127,150.119624376297,2.08125901222229,150.119624376297,1692158713,0.09219973002487042,0.06272018880860533,13881632175440.762,85051353231509.8,64,155143e0,0.13600058670157522,0.10276492262244519
FSR_Trainable_16728dc9,2023-08-16_12-56-48,True,DESKTOP-0P789CI,100,0.4609050178348739,166.8972634374087,0.1141307076370263,2.0165500155026006e+17,0.23251915322781197,172.26.215.93,34680,0.7348417166255993,303.9859906268719,240.9218134880066,2.108445882797241,240.9218134880066,1692158208,0.09236891659770471,0.06126174107208198,13631142018756.13,81567963854135.67,100,16728dc9,0.13643645263248924,0.09608270059532274
FSR_Trainable_19352681,2023-08-16_12-45-50,True,DESKTOP-0P789CI,1,0.451335014211004,237.71434971718338,0.11507515553939006,2.262088212624352e+16,6.9295849090359365,172.26.215.93,33237,0.7554366183029739,490.31867226692236,3.3022725582122803,3.3022725582122803,3.3022725582122803,1692157550,1.1673610863481758,1.7101574510062296,735513625738358.9,1134072728903929.5,1,19352681,2.2785226949749964,4.65106221406094
FSR_Trainable_1de66b93,2023-08-16_13-01-59,True,DESKTOP-0P789CI,1,0.37089797703588906,226.52434269670425,0.1033814391128331,599560028.6993338,1.8751823371272915,172.26.215.93,35854,0.6937998042440903,387.9945967908208,2.367266893386841,2.367266893386841,2.367266893386841,1692158519,0.5805764859597831,0.58612062944286,2.537603573217224,1.7709927297359034,1,1de66b93,0.9940017016800103,0.8811806354472813
FSR_Trainable_1e232507,2023-08-16_12-43-40,True,DESKTOP-0P789CI,1,0.8325458483608348,281.31012189441077,0.27619703089381076,5.019081329120868e+16,10.559701902545376,172.26.215.93,32722,1.7185691422140033,575.4934238290787,2.614701509475708,2.614701509475708,2.

2023-08-16 12:38:16,691	ERROR tune_controller.py:873 -- Trial task failed for trial FSR_Trainable_0888e0d9
Traceback (most recent call last):
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 18, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/worker.py", line 2540, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::ResourceTrainable.train() (pid=31167, ip=172.26.215.93, actor_id=5d28868d96355de7c4fc7a0601000000, repr=<ray.tune.trainable.util.FSR_Trainable object at 0x7f329e52f8b0>)
  File "/home/